Scrap and clean a list of disney wikipedia pages to create a dataset to further analyze


TASK 1 - Get infobox (Store in a python dictionary)



In [2]:
#Importing libraries

from bs4 import BeautifulSoup as bs
import requests


LOAD THE WEBPAGE

In [30]:
r = requests.get('https://en.wikipedia.org/wiki/Toy_Story_3')

# Convert to bs object
soup = bs(r.content)

contents = soup.prettify()

In [7]:
info_box = soup.find(class_ = "infobox vevent")
info_rows = info_box.findAll("tr")
for row in info_rows:
    print(row.prettify())


<tr>
 <th class="infobox-above summary" colspan="2" style="font-size: 125%; font-style: italic;">
  Toy Story 3
 </th>
</tr>

<tr>
 <td class="infobox-image" colspan="2">
  <a class="image" href="/wiki/File:Toy_Story_3_poster.jpg" title="All of the toys packed close together, holding up a large numeral '3', with Buzz, who is putting a friendly arm around Woody's shoulder, and Woody holding the top of the 3.">
   <img alt="All of the toys packed close together, holding up a large numeral '3', with Buzz, who is putting a friendly arm around Woody's shoulder, and Woody holding the top of the 3." class="thumbborder" data-file-height="326" data-file-width="220" decoding="async" height="326" src="//upload.wikimedia.org/wikipedia/en/6/69/Toy_Story_3_poster.jpg" width="220"/>
  </a>
  <div class="infobox-caption">
   Theatrical release poster
  </div>
 </td>
</tr>

<tr>
 <th class="infobox-label" scope="row" style="white-space: nowrap; padding-right: 0.65em;">
  Directed by
 </th>
 <td class="

In [29]:

def get_content_value(row_data):
    if row_data.find("li"):
        return [li.get_text(" ", strip=True).replace("\xa0", " ") for li in row_data.find_all("li")]
    else:
        return row_data.get_text(" ", strip = True).replace("\xa0", " ")
    

movie_info = {}

for index, row in enumerate(info_rows):
    if index == 0:
        movie_info['Title'] = row.find('th').get_text(" ", strip = True)
    elif index == 1:
        continue
    else:
        content_key = row.find("th").get_text(" ", strip = True)
        content_value = get_content_value(row.find("td"))
        movie_info[content_key] = content_value


GET INFOBOX FOR ALL MOVIES

In [28]:
r = requests.get('https://en.wikipedia.org/wiki/List_of_Walt_Disney_Pictures_films')

# Convert to bs object
soup = bs(r.content)

contents = soup.prettify()

In [25]:
movies = soup.select(".wikitable.sortable i")


In [56]:
def get_content_value(row_data):
    if row_data.find("li"):
        return [li.get_text(" ", strip=True).replace("\xa0", " ") for li in row_data.find_all("li")]
    elif row_data.find("br"):
        return [text for text in row_data.stripped_strings]
    else:
        return row_data.get_text(" ", strip = True).replace("\xa0", " ")


def clean_tags(soup):
    for tag in soup.find_all("sup", "span"):
        tag.decompose()

    

def get_info_box(url):
    r = requests.get(url)
    soup = bs(r.content)
    info_box = soup.find(class_ = "infobox vevent")
    info_rows = info_box.find_all("tr")

    clean_tags(soup)
    
    movie_info = {}

    for index, row in enumerate(info_rows):
        if index == 0:
            movie_info['Title'] = row.find('th').get_text(" ", strip = True)
        else:
            header = row.find('th')
            if header:
                content_key = row.find("th").get_text(" ", strip = True)
                content_value = get_content_value(row.find("td"))
                movie_info[content_key] = content_value

    return movie_info



In [57]:
get_info_box("https://en.wikipedia.org/wiki/Peter_Pan_(1953_film)")


{'Title': 'Peter Pan',
 'Directed by': ['Clyde Geronimi', 'Wilfred Jackson', 'Hamilton Luske'],
 'Story by': ['Milt Banta',
  'Bill Cottrell',
  'Winston Hibler',
  'Bill Peet',
  'Erdman Penner',
  'Joe Rinaldi',
  'Ted Sears',
  'Ralph Wright'],
 'Based on': ['Peter and Wendy', 'by', 'J. M. Barrie'],
 'Produced by': 'Walt Disney',
 'Starring': ['Bobby Driscoll',
  'Kathryn Beaumont',
  'Hans Conried',
  'Paul Collins'],
 'Narrated by': 'Tom Conway',
 'Music by': 'Oliver Wallace',
 'Production company': 'Walt Disney Productions',
 'Distributed by': 'RKO Radio Pictures',
 'Release date': ['February 5, 1953 ( 1953-02-05 ) (United States) [1]'],
 'Running time': '77 minutes',
 'Country': 'United States',
 'Language': 'English',
 'Budget': '$4 million [2]',
 'Box office': '$87.4 million (United States and Canada) [2]'}

In [58]:
r = requests.get('https://en.wikipedia.org/wiki/List_of_Walt_Disney_Pictures_films')

soup = bs(r.content)
movies = soup.select(".wikitable.sortable i a")

base_path = "https://en.wikipedia.org/"


movie_info_list = []
for index, movie in enumerate(movies):
    
    try:
        relative_path = movie['href']
        full_path = base_path + relative_path
        title = movie['title']
        movie_info_list.append(get_info_box(full_path))

    except Exception as e:
        print(movie.get_text())
        print(e)

    

Zorro the Avenger
'NoneType' object has no attribute 'find'
The Sign of Zorro
'NoneType' object has no attribute 'find'
True-Life Adventures
'NoneType' object has no attribute 'find_all'
The London Connection
'NoneType' object has no attribute 'find'
Better Nate Than Never
'NoneType' object has no attribute 'find_all'
Sister Act 3
'NoneType' object has no attribute 'find'
Tron: Ares
'NoneType' object has no attribute 'find'


SAVE / RELOAD MOVIE DATA

In [89]:
import json

def save_data(title, data):
    with open(title, 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii= False, indent= 2)

In [90]:
import json

def load_data(title):
    with open(title, encoding='utf-8') as f:
        return json.load(f)

CLEAN THE DATA

In [67]:
# Convert running time into an integer

def minutes_to_integer(running_time):
    if running_time == 'N/A':
        return None
    if isinstance(running_time, list):
        entry = running_time[0]
        return int(entry.split(" ")[0])
    else:
        return int(running_time.split(" ")[0])


for movie in movie_info_list:
    movie['Running Time (int)'] = minutes_to_integer(movie.get('Running time', "N/A"))




In [75]:
# Convert budget and box office to numbers

import re

amounts = r"thousand|million|billion"
number = r"\d+(,\d{3})*\.*\d*"
standard = fr"\${number}(-|\sto\s|-)?({number})?\s({amounts})"

def word_to_value(word):
	value_dict = {"thousand": 1000, "million": 1000000, "billion": 1000000000}
	return value_dict.get(word.lower(), 1)

def parse_word_syntax(string):
	stripped_string = string.replace(",", "")
	value = float(re.search(number, stripped_string).group())
	modifier = word_to_value(re.search(amounts, string, flags=re.I).group())
	return value*modifier

def parse_value_syntax(string):
	stripped_string = string.replace(",", "")
	return float(re.search(number, stripped_string).group())

def money_conversion(money):
	if type(money) == list:
		money = money[0]

	word_syntax = re.search(standard, money, flags=re.I)
	value_syntax = re.search(fr"\${number}", money)

	if word_syntax:
		return parse_word_syntax(word_syntax.group())
	elif value_syntax:
		return parse_value_syntax(value_syntax.group())
	else:
		return None


In [79]:
for movie in movie_info_list:
    movie['Budget (Float)'] = money_conversion(movie.get('Budget', "N/A"))
    movie['Box Office (Float)'] = money_conversion(movie.get('Box office', "N/A"))


In [86]:
# Convert dates into datetime object

from datetime import datetime

dates = [movie.get('Release date', "N/A") for movie in movie_info_list]

def clean_date(date):
    return date.split("(")[0].strip()

def date_conversion(date):
    if isinstance(date, list):
        date = date[0]

    if date == "N/A":
        return None

    date_str = clean_date(date)
    fmts = ["%B %d, %Y","%d %B %Y"]
    for fmt in fmts:
        try:
            return datetime.strptime(date_str, fmt)
        except:
            pass
    return None

for date in dates:
    date_conversion(date)

In [87]:
for movie in movie_info_list:
    movie['Release Date (datetime)'] = date_conversion(movie.get('Release date', 'N/A'))

In [140]:
# SAVE AND LOAD DATA

import pickle

def save_data_pickle(name, data):
    with open(name, 'wb') as f:
        pickle.dump(data, f)
        

In [141]:
import pickle

def load_data_pickle(name):
    with open(name, 'rb') as f:
        pickle.load(f)

In [142]:
save_data_pickle("Disney cleaned.pickle", movie_info_list)

In [153]:
a = load_data_pickle("Disney cleaned.pickle")